<a href="https://colab.research.google.com/github/joseaureliok/soulcode-martech-bootcamp-ad/blob/notebooks_aulas/aulas-resumos/05_resumo_AD2_fundamentos_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNDAMENTOS MongoDB**

Este notebook foi elaborado no Google Colaboratory e posteriormente ajustado pelo VS Code, em forma de compilação dos principais pontos do apresentados sobre o tema durante o curso.

#### Bootcamp Martech em Análise de Dados - AD2
##### SoulCode Academy
##### Professores:
                Franciane Rodrigues
                Douglas Ribeiro
                Jonatas Carneiro
##### Acadêmico:
                José Aurelio Kovalczuk de Oliveira

## **Conectando ao MongoDB**

In [ ]:
7# instalação do pymongo
!pip install -q pymongo

In [ ]:
# importação do mongoclient para conexão
from pymongo import MongoClient

In [ ]:
# configurando conexão
uri = "mongodb+srv://clusterad2.pcfvera.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
					 tls=True,
					 tlsCertificateKeyFile='/content/X509-cert-7217531531040870752.pem')

In [ ]:
# criando pasta e coleção
db = client['classmongo']
collection = db['brutos']

In [ ]:
# verificando valores no MongoDB
doc_count = collection.count_documents({})
print(doc_count)

0


## **Conectando a GCP**

In [ ]:
# instalação google services
!pip install gcsfs

In [ ]:
# importando bibliotecas principais para conexão com a bucket
import os
from google.cloud import storage
from google.colab import drive

In [ ]:
'''dados para conexão com a bucket'''
# bucket
bucket_name = 'soulcode-mongo'

# chave para conexão
serviceAccount = '/content/poised-standard-389812-25c9d2eada3b.json'

path = 'gs://soulcode-mongo/dados_brutos/csv/aula_21.07_cenipa_bruto.csv'

In [ ]:
# criação de conexão com service account do GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# conexão com a bucket
client = storage.Client()
bucket = client.get_bucket(bucket_name)

## **Integração MongoDB-Bucket**

In [ ]:
# abertura de bibliotecas de manipulação e análise
import pandas as pd
import numpy as np

### **Extração**
*Extract*

In [ ]:
# abertura dos dados da bucket em um dataframe
df = pd.read_csv(path,
                 sep=';',
                 encoding='ISO-8859-1',
                 dayfirst = True)


In [ ]:
# cópia de segurança (dados da bucket)
dfback = df.copy()

In [ ]:
# conversão em dicionário para passar os dados na coleção (collection) do MongoDB
df_dict = df.to_dict("records")
collection.insert_many(df_dict)

In [ ]:
# verificando valores no MongoDB
collection.count_documents({})

6769

In [ ]:
# imprimindo valores da coleção no MongoDB
for row in collection.find():
  print(row)

{'_id': ObjectId('64bacc5179b9e93230719407'), 'codigo_ocorrencia': 81027, 'codigo_ocorrencia1': 81027, 'codigo_ocorrencia2': 81027, 'codigo_ocorrencia3': 81027, 'codigo_ocorrencia4': 81027, 'ocorrencia_classificacao': 'INCIDENTE GRAVE', 'ocorrencia_latitude': nan, 'ocorrencia_longitude': nan, 'ocorrencia_cidade': 'SÃO PAULO', 'ocorrencia_uf': 'SP', 'ocorrencia_pais': 'BRASIL', 'ocorrencia_aerodromo': 'SBSP', 'ocorrencia_dia': '09/10/2022', 'ocorrencia_hora': '16:30:00', 'investigacao_aeronave_liberada': 'SIM', 'investigacao_status': 'ATIVA', 'divulgacao_relatorio_numero': 'A DEFINIR', 'divulgacao_relatorio_publicado': 'NÃO', 'divulgacao_dia_publicacao': nan, 'total_recomendacoes': 0, 'total_aeronaves_envolvidas': 1, 'ocorrencia_saida_pista': 'SIM'}
{'_id': ObjectId('64bacc5179b9e93230719408'), 'codigo_ocorrencia': 81030, 'codigo_ocorrencia1': 81030, 'codigo_ocorrencia2': 81030, 'codigo_ocorrencia3': 81030, 'codigo_ocorrencia4': 81030, 'ocorrencia_classificacao': 'INCIDENTE', 'ocorrenci

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### **Transformação**
*Transform*

In [ ]:
'''restaurando backup'''
#df = dfback

In [ ]:
# obtendo informações
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6769 entries, 0 to 6768
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ocorrencia               6769 non-null   int64 
 1   codigo_ocorrencia1              6769 non-null   int64 
 2   codigo_ocorrencia2              6769 non-null   int64 
 3   codigo_ocorrencia3              6769 non-null   int64 
 4   codigo_ocorrencia4              6769 non-null   int64 
 5   ocorrencia_classificacao        6769 non-null   object
 6   ocorrencia_latitude             4408 non-null   object
 7   ocorrencia_longitude            4407 non-null   object
 8   ocorrencia_cidade               6769 non-null   object
 9   ocorrencia_uf                   6765 non-null   object
 10  ocorrencia_pais                 6769 non-null   object
 11  ocorrencia_aerodromo            4151 non-null   object
 12  ocorrencia_dia                  6769 non-null   

In [ ]:
# Correção de inconsistência
df.replace(['***', '**NI'],pd.NA,inplace=True)
df.replace(pd.NA, np.NaN, inplace=True)

In [ ]:
# exclusão colunas repetidas
df =df.drop(columns = ['codigo_ocorrencia1', 'codigo_ocorrencia2','codigo_ocorrencia3', 'codigo_ocorrencia4'])

In [ ]:
# nomes das colunas
df.columns

Index(['codigo_ocorrencia', 'ocorrencia_classificacao', 'ocorrencia_latitude',
       'ocorrencia_longitude', 'ocorrencia_cidade', 'ocorrencia_uf',
       'ocorrencia_pais', 'ocorrencia_aerodromo', 'ocorrencia_dia',
       'ocorrencia_hora', 'investigacao_aeronave_liberada',
       'investigacao_status', 'divulgacao_relatorio_numero',
       'divulgacao_relatorio_publicado', 'divulgacao_dia_publicacao',
       'total_recomendacoes', 'total_aeronaves_envolvidas',
       'ocorrencia_saida_pista'],
      dtype='object')

In [ ]:
# renomeando colunas
df.rename(columns={'codigo_ocorrencia' : 'cod',
                   'ocorrencia_classificacao' : 'classificacao',
                   'ocorrencia_latitude' : 'latitude',
                   'ocorrencia_longitude' : 'longitude',
                   'ocorrencia_cidade' : 'cidade',
                   'ocorrencia_uf' : 'estado',
                   'ocorrencia_pais' : 'pais',
                   'ocorrencia_aerodromo' : 'aerodromo',
                   'ocorrencia_dia' : 'dia',
                   'ocorrencia_hora' : 'hora',
                   'investigacao_aeronave_liberada' : 'liberada',
                   'investigacao_status' : 'investigacao',
                   'divulgacao_relatorio_numero' : 'relatorio',
                   'divulgacao_relatorio_publicado' : 'publicado',
                   'divulgacao_dia_publicacao' : 'publicacao',
                   'total_recomendacoes' : 'recomendacoes',
                   'total_aeronaves_envolvidas' : 'envolvidos',
                   'ocorrencia_saida_pista' : 'saida'},
          inplace=True)

In [ ]:
# seleção de colunas úteis com drop das desnecessárias
df = df.drop(columns = ['latitude', 'longitude', 'aerodromo', 'liberada', 'relatorio', 'publicacao', 'recomendacoes'])

In [ ]:
# exclusão de linhas com valores nulos
df = df.dropna()

In [ ]:
# verificação de valores nulos por coluna
df.isna().sum()

cod              0
classificacao    0
cidade           0
estado           0
pais             0
dia              0
hora             0
investigacao     0
publicado        0
envolvidos       0
saida            0
dtype: int64

In [ ]:
#verificando tipos dos dados
df.dtypes

cod               int64
classificacao    object
cidade           object
estado           object
pais             object
dia              object
hora             object
investigacao     object
publicado        object
envolvidos        int64
saida            object
dtype: object

In [ ]:
# Transformação do tipo de dados da coluna código
df['dia'] = df['dia'].astype(np.datetime64)
df ['hora'] = df['hora'].str.split(':').str[0]

<ipython-input-68-4e476f0a65ab>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['dia'] = df['dia'].astype(np.datetime64)
<ipython-input-68-4e476f0a65ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dia'] = df['dia'].astype(np.datetime64)
<ipython-input-68-4e476f0a65ab>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df ['hora'] = df['hora'].str.split(

In [ ]:
# exibindo dados
display(df)

,cod,classificacao,cidade,estado,pais,dia,hora,investigacao,publicado,envolvidos,saida
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,2022-09-10,16,ATIVA,NÃO,1,SIM
1,81030,INCIDENTE,VITÓRIA,ES,BRASIL,2022-09-10,16,ATIVA,NÃO,1,NÃO
2,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,2022-06-10,15,FINALIZADA,NÃO,1,SIM
3,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,2022-06-10,00,FINALIZADA,NÃO,1,NÃO
4,81025,INCIDENTE,MANAUS,AM,BRASIL,2022-05-10,20,FINALIZADA,NÃO,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...
6764,40324,INCIDENTE,PELOTAS,RS,BRASIL,2010-05-01,19,FINALIZADA,NÃO,1,NÃO
6765,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,BRASIL,2010-04-01,17,FINALIZADA,SIM,1,NÃO
6766,40211,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,2010-03-01,12,FINALIZADA,NÃO,1,NÃO
6767,40349,INCIDENTE,BELÉM,PA,BRASIL,2010-03-01,11,FINALIZADA,NÃO,1,NÃO


In [ ]:
dfrest = df.copy()

### **Carregamento**
*LOAD*

#### **MongoDB**

In [ ]:
# dados tratados
ncollection = db['tratados']

In [ ]:
# conversão em dicionário para passar os dados na coleção (collection) do MongoDB
df_ndict = df.to_dict("records")
ncollection.insert_many(df_ndict)

In [ ]:
# verificando valores no MongoDB
ncollection.count_documents({})

6422

#### **Bucket**

In [ ]:
# Carregametno GCP
df.to_csv('gs://soulcode-mongo/dados_tratados/csv/entrega_cenipa.csv', index=False)

# **FIM**

Estes foram os principais fundamentos aboradados durante o curso!